# Generate qsub scripts for STEP00 operations

This notebook generates a set of PBS scripts and a bash script that submits all the PBS jobs. It works on the data analysis server of NSF NCAR: `casper.ucar.edu`.

In [14]:
import os
import yaml
import numpy as np

## Open the Config

In [15]:
config_name = os.path.realpath('verif_config.yml')

with open(config_name, 'r') as stream:
    conf = yaml.safe_load(stream)

## Set the Number of Forecasts Present

In [16]:
N_fcsts = 2894 #2*(365+366+365)
INDs = np.arange(0, N_fcsts+10, 10)
INDs[-1] = N_fcsts

## Generate the QSUB for Gathering the Forecasts

In [17]:
for i, ind_start in enumerate(INDs[:-1]):
    
    ind_end = INDs[i+1]
    
    f = open('{}gather_ForecastModel_{:03d}.sh'.format(conf['qsub']['qsub_loc'], i), 'w') 
    
    heads = '''#!/bin/bash -l

#PBS -N gather_ForecastModel
#PBS -A {project_code}
#PBS -l walltime=23:59:59
#PBS -l select=1:ncpus=4:mem=32GB
#PBS -q casper
#PBS -o gather_ForecastModel.log
#PBS -e gather_ForecastModel.err

module load conda
conda activate {conda_env}
cd {scripts_loc}
python STEP00_gather_ForecastModel.py {ind_start} {ind_end}
'''.format(project_code=conf['qsub']['project_code'],
           conda_env=conf['qsub']['conda_env'],
           scripts_loc=conf['qsub']['scripts_loc'], 
           ind_start=ind_start, 
           ind_end=ind_end)
    
    print(heads, file=f)    
    f.close()

f = open('{}step00_gather_ForecastModel_all.sh'.format(conf['qsub']['qsub_loc']), 'w')

for i, ind_start in enumerate(INDs[:-1]):
    print('qsub gather_ForecastModel_{:03d}.sh'.format(i), file=f)
    
f.close()
